# 5.2 Deep Learning Methods 

Neural networks especially autoemcoders can be effective in imputing missing values in complex dataets. 
deep learing methods particularly neural networks liek autnencodrs, offer a powerful approach for imputing missing values in comple datasets. These methods are especially useful when the data has intericate non-liner relationships that trditional statistical methdos migh not captuere effectively. 



#### Undersatnding Autoencoders for Imputation:

1. **What is an Autoencoder?**
    - An autoencoder is a type of neural network that is trained to copy its input to its output. 
    - It has a hidden layer that describes a code used to represent the input 
    - The network may be viewed as consisting of two parts: an endocer function which compressses the nput nto a latent-space representatino and a decoder function, which reconsitructs the input from the latent space.

2. **How Autoencoders Work for Imputation:
**
- The key idea is to train the autoencoder to ignore the noise (missing values) in the input data.
- During training, inputs with missing values are presented, and the network learns to predict the missing values in a way that minimizes reconstruction error for known parts of the data.
- This results in the network learning a robust representation of the data, enabling it to make reasonable guesses about missing values.
Advantages of Using Autoencoders:

3. **Handling Complex Patterns:** 
- They can capture non-linear relationships in the data, which is particularly useful for complex datasets.
- **Scalability:** They can handle large-scale datasets efficiently.
- **Flexibility:** They can be adapted to different types of data (e.g., images, text, time-series).
Implementation Considerations:

4. **Data Preprocessing:** 
- Data should be normalized or standardized before feeding it into an autoencoder.
- Network Architecture: The choice of architecture (number of layers, type of layers, etc.) depends on the complexity of the data.
- **Training Process:** It might involve techniques like dropout or noise addition to improve the model's ability to handle missing data.

5. **Example Use-Cases:**

- **Image Data:** Filling in missing pixels or reconstructing corrupted images.
- **Time-Series Data:** Imputing missing values in sequences like stock prices or weather data.
- **Tabular Data:** Handling missing entries in datasets used for machine learning.


## Implementtation Example:
Here's simplified example fo how you might set up an autoencoder for imputationin Python using TensorFlow and keras: 

In [1]:
import seaborn as sns 
import tensorflow as tf 

from tensorflow.keras.layers import Input, Dense 
from tensorflow.keras.models import Model 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder 
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline 
from sklearn.impute import SimpleImputer

In [3]:
# Load the Titanic dataset 
df_titanic = sns.load_dataset('titanic') 


In [6]:
# Selectin relevant features for simplicity 
df_titanic = df_titanic[['survived','pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]

In [7]:
# preprocessing 
#separate features aond target 
X = df_titanic.drop('survived', axis=1)
y = df_titanic['survived']

In [8]:
# Handlingmissign values and categorical variables 
numeric_features = ['age', 'fare', 'sibsp', 'parch']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

categorical_features = ['pclass', 'sex', 'embarked']
categorical_transformer = Pipeline(steps=[
    ('impter', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# ColumnTransformer for preprocessing 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features), 
        ('cat', categorical_transformer, categorical_features)
    ])



In [9]:
# preprocessing the datset 

X_preprocessed = preprocessor.fit_transform(X)

In [10]:
# splitting the dataset(we'll use the train set to train the autoencoder)
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

In [11]:
# Define the autoencoder architecture 
input_dim = X_train.shape[1]
encoding_dim =32 

In [13]:
input_layer = Input(shape =(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_layer)
decoded = Dense(input_dim, activation ='sigmoid')(encoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [14]:
# Train the atutoencoder 

autoencoder.fit(X_train, X_train, epochs=50, batch_size=256, shuffle=True, validation_split=0.2) 

Epoch 1/50

3/3 [==============================] - 1s 82ms/step - loss: 0.2258 - val_loss: 0.2237
Epoch 2/50
3/3 [==============================] - 0s 17ms/step - loss: 0.2219 - val_loss: 0.2199
Epoch 3/50
3/3 [==============================] - 0s 16ms/step - loss: 0.2181 - val_loss: 0.2162
Epoch 4/50
3/3 [==============================] - 0s 17ms/step - loss: 0.2145 - val_loss: 0.2126
Epoch 5/50
3/3 [==============================] - 0s 16ms/step - loss: 0.2109 - val_loss: 0.2091
Epoch 6/50
3/3 [==============================] - 0s 19ms/step - loss: 0.2074 - val_loss: 0.2056
Epoch 7/50
3/3 [==============================] - 0s 19ms/step - loss: 0.2040 - val_loss: 0.2022
Epoch 8/50
3/3 [==============================] - 0s 17ms/step - loss: 0.2006 - val_loss: 0.1989
Epoch 9/50
3/3 [==============================] - 0s 18ms/step - loss: 0.1974 - val_loss: 0.1955
Epoch 10/50
3/3 [==============================] - 0s 17ms/step - loss: 0.1941 - val_loss: 0.1922
Epoch 11/50
3/3 [===========

In [15]:
# Using the autoencoder for imputation on test set 
x_test_imputed = autoencoder.predict(X_test)

6/6 [==============================] - 0s 1ms/step
